# Gold Layer – Business Metrics & Aggregations

## Overview

The Gold layer represents business-level aggregations and KPIs derived from curated Silver datasets.
Unlike Bronze (raw ingestion) and Silver (cleaned & standardized data), the Gold layer is fully business-consumable, optimized for reporting, dashboards, and decision-making.

### All Gold tables are:
- Deterministic
- Aggregated
- Read-optimized
- Free of ingestion or technical metadata

## Source Tables
### Silver Orders
- Catalog: `migration_project_db_ws`
- Schema: `silver`
- Table: `orders`
```sql
migration_project_db_ws.silver.orders
```

---

### Silver Customers
- Catalog: `migration_project_db_ws`
- Schema: `silver`
- Table: `customers`

```sql
migration_project_db_ws.silver.customers
```

## Gold Tables Produced
| Gold Table                  | Business Metric           |
| --------------------------- | ------------------------- |
| `gold.daily_sales`          | Revenue per day           |
| `gold.revenue_per_customer` | Customer lifetime revenue |
| `gold.daily_sales_country`  | Daily revenue by country  |


## Processing Type
- Batch aggregation
- Fully derived from Silver layer
- Recomputable and idempotent

## Gold Metric 1 -- Daily Revenue
### Business Question
How much revenue do we generate per day?

### Target Table
- Catalog: `migration_project_db_ws`
- Schema: `gold`
- Table: `daily_sales`

```sql
migration_project_db_ws.gold.daily_sales
```

## Transformation Logic

### Step 1 -- Read Silver Orders
```sql
silver_orders_df = spark.read.table("silver.orders")
```
---

### Step 2 - Aggregate Daily Sales
```python
from pyspark.sql.functions import sum

gold_daily_sales_df = (
    silver_orders_df
    .groupBy("order_date")
    .agg(
        sum("amount").alias("daily_sales")
    )
    .orderBy("order_date")
)
```
---

#### Applied logic:
- Group by business date
- Aggregate monetary values
- Produce one row per day

---

## Step 3 -- Write to Gold Table
```python
gold_daily_sales_df.write \
    .mode("overwrite") \
    .saveAsTable("gold.daily_sales")
```

## Validation
```sql
SELECT * FROM gold.daily.sales ORDER_BY order_date;
```

## Business Usage
- Execute revenue tracking
- Daily trend analysis
- Finance and sales dashboards

## Gold Metric 2 -- Revenue per Customer
### Business Question
#### Which customer generate the most revenue?




### Target Table
- Catalog: `migration_project_db_ws`
- Schema: `gold`
- Table: `revenue_per_customer`
```sql
migration_project_db_ws.gold.revenue_per_customer
```

### Transformation Logic
#### Step 1 -- Aggregate Revenue by Customer
```python
from pyspark.sql.functions import sum

gold_revenue_per_customer_df = (
    silver_orders_df
    .groupBy("customer_id")
    .agg(
        sum("amount").alias("total_revenue")
    )
    .orderBy("total_revenue", ascending=False)
)
```
### Applied logic:
  - Aggregate across full order history
  - Rank customers by revenue contribution

#### Step 2 -- Write the Gold Table
```python
gold_revenue_per_customer_df.write \
    .mode("overwrite") \
    .saveAsTable("gold.revenue_per_customer")
```

## Validation
```sql
SELECT * FROM gold.revenue_per_customer;
```

### Business Usage
- Customer segmentation
- Loyalty & retention programs
- Sales prioritization

---

## 

# Gold Metric 3 -- Daily Revenue by Country (Enriched Metric)
## Business Question
### How does daily revenue vary across countries?

> This metric demonstrates cross-domain enrichment by combining transactional and master data

## Target Table
- Catalog: `migration_project_db_ws`
- Schema: `gold`
- Table: `daily_sales_country`

```sql
migration_project_db_ws.gold.daily_sales_country
```

## Transformation Logic
### Step 1 -- Read Silver Orders & Customers
```python
silver_orders_df = spark.read.table("silver.orders")
silver_customers_df = spark.read.table("silver.customers")
```

### Step 2 -- Join & Aggregate
```python
from pyspark.sql.functions import sum

gold_daily_sales_country_df = (
    silver_orders_df.alias("o")
    .join(
        silver_customers_df.alias("c"),
        on="customer_id",
        how="inner"
    )
    .groupBy("order_date", "country")
    .agg(
        sum("amount").alias("daily_sales")
    )
    .orderBy("order_date", "country")
)
```

### Key concepts demonstrated:
- Dimensional enrichment
- Fact-to-dimension joins
- Multi-dimensional aggregation


### Step 3 -- Write to Gold Table
```python
gold_daily_sales_country_df.write \
    .mode("overwrite") \
    .saveAsTable("gold.daily_sales_country")
```


## Validation
```sql
SELECT * FROM gold.daily_sales_country;
```

### Business Usage
- Regional performance analysis
- Coutry-level forecasting
- Strategic expansion decisions

## Production Alignment Note
### In production:
- Gold tables power BI tools (Power BI, Tableau)
- Incremental refresh strategies may be applied
- Access is restricted to business users
- SLAs are defined around Gold availability

## Summary
- Gold layer answers explicit business questions
- Data is aggregated, enriched, and analytics-ready
- Clean separation from ingestion and transformation logic
- Strong demonstration of end-to-end lakehouse design